**Sample dataset code by the authors modified to download images and masks**

In [ ]:
import collections
import copy
import numpy as np
import json
import os
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import measurements
import skimage.exposure
from sklearn import linear_model
from typing import Any, Dict, Iterable, List, Mapping, Optional, Tuple
import pandas as pd
from matplotlib import pyplot as plt
import scipy.misc
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
import gdal
import gdalconst

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,105 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,079 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.dow

In [ ]:
def path_from_filename(landsat_filename):
  """Returns the full GCP filepath for the provided `landsat_filename`.

  An example filename looks like:
  LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  The returned path ends up looking like:
  gs://gcp-public-data-landsat/LC08/01/221/074/LC08_L1TP_221074_20181218_20181227_01_T1/LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  Args:
    landsat_filename: str, file name of a landsat scene.
  Returns:
    str, full path to the corresponding landsat scene.
  """
  split_name = landsat_filename.split('_')
  path, row = split_name[2][:3], split_name[2][3:]
  bands = split_name[0]  # Either LC08 or LT08
  parent_dir = '_'.join(split_name[:-1])
  return os.path.join(f'gs://gcp-public-data-landsat/{bands}/01/',
                      path, row, parent_dir, landsat_filename)
  


In [ ]:
j=0
import matplotlib.image

In [ ]:
import collections
def raster_image(tif, downsample_factor=10):
      """Returns the raster images, possibly downsampled.

      Args:
        tif: GeoTIFF image of landsat band.
        downsample_factor: float or int, how many times smaller to downsample to.

      Returns:
        Rastered version of the band.
      """
      # First read is just to get shape, so we can do the second read properly.
      data = tif.ReadAsArray()
      data = tif.ReadAsArray(
          buf_xsize=int(data.shape[1] / downsample_factor),
          buf_ysize=int(data.shape[0] / downsample_factor),
          resample_alg=gdalconst.GRIORA_Average)
      data = np.where(data == 0, np.nan, data)
      return data


def parse_metadata(path_mtl):
          """Returns parsed metadata.

          Args:
            path: path to a local MTL.txt file.

          Raises:
            OSError: if the metadata is missing.

          Returns:
            metadata dict
          """
          if not os.path.exists(path_mtl):
            raise OSError('Missing file: ' + path_mtl)

          metadata = {}
          with open(path_mtl) as f:
            for line in f:
              if ' = ' in line:
                split_line = line.split(' = ')
                try:
                  metadata[split_line[0].strip()] = float(split_line[-1].strip())
                except ValueError:
                  metadata[split_line[0].strip()] = split_line[-1].strip()
          return metadata
        
BandConstants = collections.namedtuple('BandConstants',
                                              ['mult', 'add', 'k1', 'k2'])
RADIANCE_ADD_KEY = 'RADIANCE_ADD_BAND_'
RADIANCE_MULT_KEY = 'RADIANCE_MULT_BAND_'
REFLECTANCE_ADD_KEY = 'REFLECTANCE_ADD_BAND_'
REFLECTANCE_MULT_KEY = 'REFLECTANCE_MULT_BAND_'
K1_KEY = 'K1_CONSTANT_BAND_'
K2_KEY = 'K2_CONSTANT_BAND_'


def read_band_constants(metadata,
                                band,
                                reflectance=True):
          """Extract band constants from Landsat Metadata.

          Args:
            metadata: Metadata from the LandSat file.
            band: name of the band to get constants for.
            reflectance: if True, read constants for reflectance rather than
              temperature.

          Returns:
            A BandConstants object containing the constants.
          """
          if reflectance:
            return BandConstants(metadata[REFLECTANCE_MULT_KEY + band],
                                metadata[REFLECTANCE_ADD_KEY + band],
                                None, None)
          else:
            return BandConstants(metadata[RADIANCE_MULT_KEY + band],
                                metadata[RADIANCE_ADD_KEY + band],
                                metadata[K1_KEY + band],
                                metadata[K2_KEY + band])  


def top_of_atmosphere_temperature(image, band_constants):
            """Conversion of radiances to brightness temperature.

            Described here:
            https://www.usgs.gov/core-science-systems/nli/landsat/using-usgs-landsat-level-1-data-product

            Args:
              image: np.ndarray, radiance to convert to brightness temperature.
              band_constants: constants needed by the conversion.

            Returns:
              np.ndarray of brightness temperature (units of K).
            """
            radiance = image * band_constants.mult
            radiance += band_constants.add
            denom = np.log((band_constants.k1 / radiance) + 1.)
            return band_constants.k2 / denom


def reflectance(image, band_constants):
            return image * band_constants.mult + band_constants.add    


def normalize_for_rgb(signal,
                              bounds,
                              adapt=True) -> np.ndarray:
          """Map an array to (0, 1)."""
          start, end = bounds
          out = ((signal - start) / (end - start)).clip(0, 1)
          if (np.all(np.isnan(out) | np.isclose(out, 0, atol=1e-3)) or
              np.all(np.isnan(out) | np.isclose(out, 1, atol=1e-3))):
            adapt = False
          if adapt:
            out = skimage.exposure.equalize_adapthist(out, clip_limit=0.03)
          return out

def truecolor_rgb(path_red, path_green, path_blue):
          """Given Landsat red, green & blue filepaths, return an image for plotting."""
          
          tif_red = gdal.Open(path_red)
          tif_green = gdal.Open(path_green)
          tif_blue = gdal.Open(path_blue)

          radiance_red = raster_image(tif_red)
          radiance_green = raster_image(tif_green)
          radiance_blue = raster_image(tif_blue)

          del tif_red
          del tif_green
          del tif_blue

          rgb = np.stack([radiance_red, radiance_green, radiance_blue], axis=-1)
          # Landsat gives us the radiance as a 16-bit integer, convert to
          # float in (0, 1).
          rgb = rgb / (2**16 - 1)
          # This routine increases the contrast on dark/bright parts of the image.
          for i in range(3):
            rgb[..., i] = skimage.exposure.equalize_adapthist(
                rgb[..., i], clip_limit=0.1)
          return rgb
def plot(image, ax):
          ax.imshow(image)
          ax.set_xticks([])
          ax.set_yticks([])


In [ ]:
 def get_false_color_image(path_11um, path_12um, path_1370nm, path_mtl, night=False):
        tif_11um = gdal.Open(path_11um)
        tif_12um = gdal.Open(path_12um)
        tif_1370nm = gdal.Open(path_1370nm)

        radiance_11um = raster_image(tif_11um)
        radiance_12um = raster_image(tif_12um)
        radiance_1370nm = raster_image(tif_1370nm)

        del tif_11um
        del tif_12um
        del tif_1370nm

        metadata = parse_metadata(path_mtl)
        constants_11um = read_band_constants(metadata, '10', reflectance=False)
        constants_12um = read_band_constants(metadata, '11', reflectance=False)
        constants_1370nm = read_band_constants(metadata, '9', reflectance=True)  

        temperature_11um = top_of_atmosphere_temperature(radiance_11um, constants_11um)
        temperature_12um = top_of_atmosphere_temperature(radiance_12um, constants_12um)
        reflectance_1370nm = reflectance(radiance_1370nm, constants_1370nm)

        tdiff = temperature_11um - temperature_12um

        # The red band is negative of the brightness temperature difference
        # (so 1 means no brightness temperature difference, 0 means large
        # brightness temperature difference)
        tdiff_clip = (-1, 5.5)  
        tdiff_bounds = (-tdiff_clip[1], -tdiff_clip[0])
        ir_r = normalize_for_rgb(-1 * tdiff, tdiff_bounds, adapt=False)

        # The green band is 1 - cirrus reflectance. So zero means the cirrus band
        # is highly reflective (indicating the presence of cirrus clouds), 1 means
        # the cirrus band is not reflective.
        if night:
          ir_g = np.zeros_like(reflectance_1370nm)
          t12_bounds = (243, 303)  # From a sample of nighttime landsat scenes.
        else:
          ir_g = normalize_for_rgb(1 - reflectance_1370nm, bounds=(0.8, 1))
          t12_bounds = (283, 303)  # From a sample of daytime landsat scenes.

        # The blue band is the 12 um brightness temperature. Zero means cold,
        # (which is consistent with contrails), one means hot, so probably not a
        # contrail.
        ir_b = normalize_for_rgb(temperature_12um, t12_bounds)
        
        return np.stack([ir_r, ir_g, ir_b], axis=-1), temperature_11um, temperature_12um, reflectance_1370nm

In [ ]:
i=0

In [ ]:
base_filename2=0
for z in range(0,100):
  !rm {os.path.join('/tmp/', base_filename2)}
  dir_name='gs://landsat_contrails_dataset/2021_06_11_1623455786/data'
  if z<10:
    base_filename1=f'landsat_contrails.json-0000{z}-of-00100 '
    base_filename2=f'/tmp/landsat_contrails.json-0000{z}-of-00100'
  else:
    base_filename1=f'landsat_contrails.json-000{z}-of-00100 '
    base_filename2=f'/tmp/landsat_contrails.json-000{z}-of-00100'

  FILE_NAME=os.path.join(dir_name, base_filename1+base_filename2)
  #FILE_NAME = 'gs://landsat_contrails_dataset/2021_06_11_1623455786/data/landsat_contrails.json-00000-of-00100 /tmp/landsat_contrails.json-00000-of-00100 '
  print(FILE_NAME)

  !gsutil cp {FILE_NAME}
  #!gsutil cp gs://landsat_contrails_dataset/2021_06_11_1623455786/data /tmp/landsat_contrails.json-00000-of-00100 
  with open(base_filename2) as f:
  #with open('/tmp/landsat_contrails.json-00000-of-00100') as f:
    for jsonObj in f:
      scene_data = json.loads(jsonObj)
      filename_11um=scene_data['filename']
      filename_12um = filename_11um.replace('B10.TIF', 'B11.TIF')
      filename_1370nm = filename_11um.replace('B10.TIF', 'B9.TIF')
      filename_red =  filename_11um.replace('B10.TIF', 'B4.TIF')
      filename_green =  filename_11um.replace('B10.TIF', 'B3.TIF')
      filename_blue =  filename_11um.replace('B10.TIF', 'B2.TIF')
      filename_mtl = filename_11um.replace('B10.TIF', 'MTL.txt')

      !gsutil cp {path_from_filename(filename_11um)} /tmp/
      !gsutil cp {path_from_filename(filename_12um)} /tmp/
      !gsutil cp {path_from_filename(filename_1370nm)} /tmp/
      !gsutil cp {path_from_filename(filename_red)} /tmp/
      !gsutil cp {path_from_filename(filename_green)} /tmp/
      !gsutil cp {path_from_filename(filename_blue)} /tmp/
      !gsutil cp {path_from_filename(filename_mtl)} /tmp/

      path_11um = os.path.join('/tmp/', filename_11um)
      path_12um = os.path.join('/tmp/', filename_12um)
      path_1370nm = os.path.join('/tmp/', filename_1370nm)
      path_red = os.path.join('/tmp/', filename_red)
      path_green = os.path.join('/tmp/', filename_green)
      path_blue = os.path.join('/tmp/', filename_blue)
      path_mtl = os.path.join('/tmp/', filename_mtl)
      plt.figure(figsize=(20, 20))

     

      false_color_image, temperature_11um, temperature_12um, reflectance_1370nm = get_false_color_image(path_11um, path_12um, path_1370nm, path_mtl)
      ax = plt.subplot()
      plot(false_color_image, ax)
      name='/content/drive/MyDrive/Contrails_Dataset/False_Colour/1fci'+str(i)+'.jpg'
      plt.gca().set_axis_off()
      plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
      plt.margins(0,0)
      plt.gca().xaxis.set_major_locator(plt.NullLocator())
      plt.gca().yaxis.set_major_locator(plt.NullLocator())

      plt.savefig(name, bbox_inches='tight',pad_inches=0,edgecolor ='White')


    
      #Plot and save False Colour Images 
      plot(false_color_image, ax)
      name='/content/drive/MyDrive/Contrails_Dataset/FCI/fci'+str(i)+'.jpg'
      plt.gca().set_axis_off()
      plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
      plt.margins(0,0)
      plt.gca().xaxis.set_major_locator(plt.NullLocator())
      plt.gca().yaxis.set_major_locator(plt.NullLocator())
      plt.savefig(name, bbox_inches='tight',pad_inches=0,edgecolor ='White')
      ax.clear()
    
      #create np.zeros() array of the same shape as rgb for the contrail mask
      rgb = truecolor_rgb(path_red, path_green, path_blue)
      sh=rgb.shape
      # print(sh)
      a=np.zeros(sh)

      #Plot contrail's bounding polygons and save as masks
      patches = []
      for polygon in scene_data['polygons']:
        patches.append(matplotlib.patches.Polygon(np.array(polygon), True, color=[1, 1, 1]))
      p = matplotlib.collections.PatchCollection(patches, alpha=1, match_original=True)
      ax.add_collection(p)
      plot(a, ax)
      name='/content/drive/MyDrive/Contrails_Dataset/Contrail/contrail'+str(i)+'.jpg'
      plt.gca().set_axis_off()
      plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
      plt.margins(0,0)
      plt.gca().xaxis.set_major_locator(plt.NullLocator())
      plt.gca().yaxis.set_major_locator(plt.NullLocator())
      plt.savefig(name, bbox_inches='tight',pad_inches=0,edgecolor ='White')

   
      #Plot and download RGB Images
      # bx = plt.subplot()
      ax.clear()
      plot(rgb, ax)
    # print(rgb.shape)
      name='/content/drive/MyDrive/Contrails_Dataset/RGB/rgb'+str(i)+'.jpg'
      plt.gca().set_axis_off()
      plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
      plt.margins(0,0)
      plt.gca().xaxis.set_major_locator(plt.NullLocator())
      plt.gca().yaxis.set_major_locator(plt.NullLocator())
      plt.savefig(name, bbox_inches='tight',pad_inches=0,edgecolor ='White')
      i+=1


      !rm {path_11um}
      !rm {path_12um}
      !rm {path_1370nm}
      !rm {path_red}
      !rm {path_green}
      !rm {path_blue}
      !rm {path_mtl}

In [ ]:
print(i)